In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
df = pd.read_parquet("https://osf.io/gk2ty/download")


In [ ]:
df["Components"] = df["Task"].map(
    {
        "AND": 2,
        "ANDNOT": 3,
        "NAND": 1,
        "NOR": 4,
        "NOT": 1,
        "OR": 3,
        "ORNOT": 2,
        "XOR": 4,
        "EQUALS": 5,
    },
)


In [ ]:
dfx = df.groupby(
    ["Generation Born", "Components", "Run ID", "Treatment"],
    observed=True,
).agg(
    **{
        "Delta Has Task": ("Delta Has Task", "first"),
        "SLIP_INSERTION_BOOL_MASK": ("SLIP_INSERTION_BOOL_MASK", "max"),
        "POINT_MUTATION_BOOL_MASK": ("POINT_MUTATION_BOOL_MASK", "max"),
    },
)


In [ ]:
del df


In [ ]:
dfy = dfx.reset_index()
dfy


In [ ]:
tp.tee(
    sns.barplot,
    data=dfy[
        dfy["Treatment"] == "Slip-duplicate"
    ].astype(
        {
            "Delta Has Task": "category",
            "SLIP_INSERTION_BOOL_MASK": "bool",
            "POINT_MUTATION_BOOL_MASK": "category",
        }
    ),
    x="Components",
    y="SLIP_INSERTION_BOOL_MASK",
    hue="Delta Has Task",
)


In [ ]:
saveit, result = tp.tee(
    sns.displot,
    data=dfy[
        dfy["Treatment"] == "Slip-duplicate"
    ].astype(
        {
            "Delta Has Task": "category",
            "SLIP_INSERTION_BOOL_MASK": "category",
            "POINT_MUTATION_BOOL_MASK": "category",
        }
    ),
    col="Components",
    col_wrap=3,
    x="Delta Has Task",
    kind="hist",
    stat="count",
    multiple="dodge",
    hue="SLIP_INSERTION_BOOL_MASK",
    discrete=True,
    teeplot_callback = True,
    shrink=0.8,
)
plt.yscale('log')

# add count labels
# adapted from https://stackoverflow.com/a/55319634/17332200
for ax in result.axes.flat:
    for p in ax.patches:
        ax.annotate(
            text=f"{p.get_height():1.0f}",
            xy=(p.get_x() + p.get_width() / 2., p.get_height()),
            xycoords='data',
            ha='center',
            va='center',
            fontsize=11,
            color='black',
            xytext=(0,7),
            textcoords='offset points',
            clip_on=True,                   # <---  important
        )

saveit()


In [ ]:
sns.displot(
    data=dfx.astype(
        {
            "Delta Has Task": "category",
            "SLIP_INSERTION_BOOL_MASK": "category",
            "POINT_MUTATION_BOOL_MASK": "category",
        }
    ),
    x="Delta Has Task",
    hue="SLIP_INSERTION_BOOL_MASK",
    col="Components",
    kind="hist",
    multiple="fill",
    row="Treatment",
    facet_kws=dict(margin_titles=True, ),
    aspect=0.8,
    height=2.5,
).set_titles(row_template="{row_name}", col_template="{col_name}")


In [ ]:
g = sns.displot(
    data=dfx.astype(
        {
            "Delta Has Task": "category",
            "SLIP_INSERTION_BOOL_MASK": "category",
            "POINT_MUTATION_BOOL_MASK": "category",
        }
    ),
    x="Delta Has Task",
    hue="SLIP_INSERTION_BOOL_MASK",
    col="Components",
    kind="hist",
    multiple="dodge",
    row="Treatment",
    facet_kws=dict(margin_titles=True, ),
    aspect=0.8,
    height=2.5,
    shrink=0.8,
).set_titles(row_template="{row_name}", col_template="{col_name}")

for ax in g.axes.flat:
    ax.set_yscale("symlog")
